[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/ebgv/Planet--Understanding-the-Amazon-from-Space/blob/master/building_model.ipynb)

In [2]:
# check the environment 

%ls

data/  sample_data/


In [3]:
# check if the data is in the environment 

%cd /content/data/sample/test/cloudy
%ls

/content/data/sample/test/cloudy
train_13312.jpg  train_22371.jpg  train_28309.jpg  train_6465.jpg
train_15915.jpg  train_2424.jpg   train_34531.jpg  train_6694.jpg
train_18976.jpg  train_25952.jpg  train_38947.jpg  train_7308.jpg
train_19000.jpg  train_26540.jpg  train_4214.jpg   train_8519.jpg
train_20914.jpg  train_27921.jpg  train_5032.jpg   train_9300.jpg


In [4]:
# check if the data is in the environment 

%cd /content/data/sample/train/cloudy
%ls

/content/data/sample/train/cloudy
train_10663.jpg  train_19130.jpg  train_31869.jpg  train_35987.jpg
train_10687.jpg  train_22569.jpg  train_32283.jpg  train_36363.jpg
train_11294.jpg  train_24328.jpg  train_32513.jpg  train_37350.jpg
train_13248.jpg  train_26341.jpg  train_33291.jpg  train_38968.jpg
train_17009.jpg  train_28686.jpg  train_33639.jpg  train_40475.jpg


# Imports 

In [5]:
# to install pytorch on colab
from os import path
from wheel.pep425tags import get_abbr_impl, get_impl_ver, get_abi_tag
platform = '{}{}-{}'.format(get_abbr_impl(), get_impl_ver(), get_abi_tag())

accelerator = 'cu80' if path.exists('/opt/bin/nvidia-smi') else 'cpu'

!pip install -q http://download.pytorch.org/whl/{accelerator}/torch-0.4.1-{platform}-linux_x86_64.whl torchvision

tcmalloc: large alloc 1073750016 bytes == 0x563f4000 @  0x7fb705a872a4 0x594e17 0x626104 0x51190a 0x4f5277 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x510c78 0x5119bd 0x4f5277 0x4f3338 0x510fb0 0x5119bd 0x4f6070 0x4f3338 0x510fb0 0x5119bd 0x4f6070


In [6]:
!pip install -U bcolz

    100% |████████████████████████████████| 1.5MB 14.7MB/s 
  Running setup.py bdist_wheel for bcolz ... - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ | / - \ done
  Stored in directory: /root/.cache/pip/wheels/9f/78/26/fb8c0acb91a100dc8914bf236c4eaa4b207cb876893c40b745
Successfully built bcolz


In [7]:
!pip install Pillow==4.0.0

    100% |████████████████████████████████| 5.6MB 6.7MB/s 
torchvision 0.2.1 has requirement pillow>=4.1.1, but you'll have pillow 4.0.0 which is incompatible.
  Found existing installation: Pillow 5.3.0
    Uninstalling Pillow-5.3.0:
      Successfully uninstalled Pillow-5.3.0


In [0]:
from PIL import Image
import numpy as np
import matplotlib.pyplot as plt
import os
import torch
import torch.nn as nn
import torchvision
from torchvision import models,transforms,datasets
import bcolz
import time

# GPU Settings 

In [9]:
use_gpu = torch.cuda.is_available()
print('Using gpu: %s ' % use_gpu)

dtype = torch.FloatTensor
if use_gpu:
    dtype = torch.cuda.FloatTensor

Using gpu: True 


# Data Loader

In [0]:
# loader taken from class example: parameters to verify

def save_array(fname, arr):
    c=bcolz.carray(arr, rootdir=fname, mode='w')
    c.flush()
def load_array(fname):
    return bcolz.open(fname)[:]

normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]) # statistics from imagenet ? 

prep1 = transforms.Compose([
                transforms.CenterCrop(224), # default cropping 
                transforms.ToTensor(),
                normalize,
            ])

In [0]:
data_dir = '/content/data/sample'
#data_dir = '/content/data'

batch_size = 4
#batch_size = 64

In [0]:
# dictionary of datasets - in this case only train data set 
dsets = {x: datasets.ImageFolder(os.path.join(data_dir, x), prep1)
         for x in ['train', 'test']}

In [13]:
# train labels 
print(dsets['train'].imgs[:5]) # 5 first images and labels 
print(dsets['train'].imgs[len(dsets['train'])-5:]) # 5 last images and labels 

[('/content/data/sample/train/clear/train_10162.jpg', 0), ('/content/data/sample/train/clear/train_10190.jpg', 0), ('/content/data/sample/train/clear/train_10463.jpg', 0), ('/content/data/sample/train/clear/train_12265.jpg', 0), ('/content/data/sample/train/clear/train_17530.jpg', 0)]
[('/content/data/sample/train/partly_cloudy/train_5260.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_7695.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_8991.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9348.jpg', 3), ('/content/data/sample/train/partly_cloudy/train_9581.jpg', 3)]


In [14]:
# test labels 
print(dsets['test'].imgs[:5]) # 5 first images and labels 
print(dsets['test'].imgs[len(dsets['test'])-5:]) # 5 last images and labels 

[('/content/data/sample/test/clear/train_11403.jpg', 0), ('/content/data/sample/test/clear/train_11708.jpg', 0), ('/content/data/sample/test/clear/train_14992.jpg', 0), ('/content/data/sample/test/clear/train_16269.jpg', 0), ('/content/data/sample/test/clear/train_22239.jpg', 0)]
[('/content/data/sample/test/partly_cloudy/train_8311.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_8746.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9150.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9278.jpg', 3), ('/content/data/sample/test/partly_cloudy/train_9707.jpg', 3)]


In [15]:
dset_classes = dsets['train'].classes
dset_classes # binary classes to begin 

['clear', 'cloudy', 'haze', 'partly_cloudy']

In [0]:
# dictionary of data loaders - again only train for now 
dset_loaders = {x: torch.utils.data.DataLoader(dsets[x], batch_size=batch_size,
                                               shuffle=False, num_workers=0)
                for x in ['train', 'test']}

In [17]:
print(len(dsets['train']))
print(len(dsets['test']))

80
80


# Model: simple classifier from scratch

In [0]:
# need to understand parameters in more details !!!!

import torch.nn as nn
import torch.nn.functional as F # other way to define layers: when no need to update parameters 

# max pooling layer is not in init: no parameters to update ie. deterministic function  

class classifier(nn.Module):
    
    def __init__(self):
        super(classifier, self).__init__() # do the init from the inherited class
        # then what are the parameters of the network: need to be defined in init 
        # fill the missing entries below
        self.conv1 = nn.Conv2d(in_channels=3, out_channels=64, kernel_size=3, padding=1) # check meaning of arguments  
        self.fc = nn.Linear(in_features=32*32*64, out_features=4) # 32 = 224 / 7
        
    def forward(self,x):
        # implement your network here, use F.max_pool2d, F.log_softmax and do not forget to flatten your vector
        x = self.conv1(x)
        x = F.max_pool2d(x, kernel_size=7, stride=7)
        x = x.view(-1, 32*32*64) # flatten 
        x = self.fc(x)
        return F.log_softmax(x, dim=1) # softmax across the line ! not the component 
    
# since we inherit from nn module, the forward is called by default (do not call classifier.forward() !!

# Model vgg16 by hand 

In [0]:
import torch
import torch.nn as nn
import torch.nn.functional as F


class VGG_16(nn.Module):

    def __init__(self,num_classes=4):
        super(VGG_16, self).__init__()
        self.convblock1 = nn.Sequential(
            nn.Conv2d(3, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(64, 64, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.convblock2 = nn.Sequential(
            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(128, 128, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.convblock3 = nn.Sequential(
            nn.Conv2d(128, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(256, 256, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.convblock4 = nn.Sequential(
            nn.Conv2d(256, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.convblock5 = nn.Sequential(
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.Conv2d(512, 512, kernel_size=3, padding=1),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
        )
        self.classifier = nn.Sequential(
            nn.Linear(512 * 7 * 7, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, 4096),
            nn.ReLU(True),
            nn.Dropout(),
            nn.Linear(4096, num_classes),
        )       

    def forward(self, x):
        x = self.convblock1(x)
        x = self.convblock2(x)
        x = self.convblock3(x)
        x = self.convblock4(x)
        x = self.convblock5(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x
    
    def init_weight(self,w):
        i=0
        for idx, m in enumerate(self.children()):
            for idy, msub in enumerate(m.children()):
                classname = msub.__class__.__name__
                if classname.find('Conv') != -1:
                    msub.weight.data = w['features.'+str(i)+'.weight']#.clone()
                    msub.bias.data = w['features.'+str(i)+'.bias']#.clone()
                    print(msub,'features.'+str(i))
                if classname.find('Linear') != -1:
                    msub.weight.data = w['classifier.'+str(i-31)+'.weight']#.clone()
                    msub.bias.data = w['classifier.'+str(i-31)+'.bias']#.clone()
                    print(msub,'classifier.'+str(i-31))
                i +=1

# Training Framework

In [0]:
def train(model,data_loader,loss_fn,optimizer,n_epochs=1):
    
    model.train(True)
    
    loss_train = np.zeros(n_epochs)
    acc_train = np.zeros(n_epochs)
    
    for epoch_num in range(n_epochs):
        running_corrects = 0.0
        running_loss = 0.0
        size = 0

        for data in data_loader:
            inputs, labels = data
            if use_gpu:
                inputs, labels = inputs.cuda(), labels.cuda()    
                
            # batch_size ?
            bs = labels.size(0)
            
            # define the loss to minimize
            outputs = model(inputs)
            loss = loss_fn(outputs, labels)
            # define the optimizer
            optimizer = optimizer
            optimizer.zero_grad()
            # backpropagation
            loss.backward()
            optimizer.step()
            
            # predictions to get statistics 
            _,preds = torch.max(outputs.data,1)
            # statistics
            running_loss += loss.data.item()
            running_corrects += torch.sum(preds == labels.data)
            size += bs
        # epoch statistics     
        epoch_loss = running_loss / size
        epoch_acc = running_corrects.item() / size
        loss_train[epoch_num] = epoch_loss
        acc_train[epoch_num] = epoch_acc
        print('Train - Loss: {:.4f} Acc: {:.4f}'.format(epoch_loss, epoch_acc))
        
    return loss_train, acc_train

# Running training epochs

# Using vgg 16 by hand 

In [30]:
# instanciate the model 
net = VGG_16()
use_gpu = torch.cuda.is_available()
if use_gpu:
    net = net.cuda()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.Adam(conv_class.parameters(), lr=learning_rate)


# and train for 10 epochs
l_t, a_t = train(net, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 10)

Train - Loss: 0.3465 Acc: 0.2750
Train - Loss: 0.3465 Acc: 0.2875
Train - Loss: 0.3467 Acc: 0.2375
Train - Loss: 0.3465 Acc: 0.2500
Train - Loss: 0.3469 Acc: 0.2000
Train - Loss: 0.3466 Acc: 0.2250
Train - Loss: 0.3466 Acc: 0.2500
Train - Loss: 0.3464 Acc: 0.3250
Train - Loss: 0.3464 Acc: 0.2625
Train - Loss: 0.3465 Acc: 0.2375


In [31]:
# train for 10 more epochs
l_t, a_t = train(conv_class, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 10)

Train - Loss: 2.5555 Acc: 0.6000
Train - Loss: 8.0829 Acc: 0.4875
Train - Loss: 12.3866 Acc: 0.3375
Train - Loss: 3.2874 Acc: 0.5750
Train - Loss: 1.4837 Acc: 0.6500
Train - Loss: 0.3237 Acc: 0.8625
Train - Loss: 0.5194 Acc: 0.8500
Train - Loss: 0.3695 Acc: 0.8250
Train - Loss: 2.0836 Acc: 0.6375
Train - Loss: 1.0061 Acc: 0.7125


# Using the simple classfier 

In [24]:
# using the model from scratch

# instanciate the model 
conv_class = classifier()
use_gpu = torch.cuda.is_available()
if use_gpu:
    conv_class = conv_class.cuda()

# choose the appropriate loss
loss_fn = torch.nn.CrossEntropyLoss()
# learning rate 
learning_rate = 1e-3
# your SGD optimizer
optimizer_cl = torch.optim.Adam(conv_class.parameters(), lr=learning_rate)


# and train for 10 epochs
l_t, a_t = train(conv_class, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 10)

Train - Loss: 4.3163 Acc: 0.4625
Train - Loss: 7.1527 Acc: 0.2250
Train - Loss: 1.4121 Acc: 0.6000
Train - Loss: 3.8656 Acc: 0.4500
Train - Loss: 0.9734 Acc: 0.6000
Train - Loss: 1.1560 Acc: 0.5875
Train - Loss: 0.2495 Acc: 0.8000
Train - Loss: 0.5856 Acc: 0.6500
Train - Loss: 0.1370 Acc: 0.8625
Train - Loss: 0.7864 Acc: 0.5500


In [25]:
# train for 10 more epochs
l_t, a_t = train(conv_class, dset_loaders['train'], loss_fn, optimizer_cl, n_epochs = 10)

Train - Loss: 1.0411 Acc: 0.5500
Train - Loss: 0.2283 Acc: 0.7875
Train - Loss: 0.2435 Acc: 0.7875
Train - Loss: 0.3782 Acc: 0.7125
Train - Loss: 0.0925 Acc: 0.9000
Train - Loss: 0.3301 Acc: 0.7625
Train - Loss: 0.1617 Acc: 0.9500
Train - Loss: 0.0409 Acc: 0.9500
Train - Loss: 0.1115 Acc: 0.8875
Train - Loss: 0.0597 Acc: 0.9500


# Testing 

In [0]:
def test(model,data_loader):
    model.train(False)

    running_corrects = 0.0
    running_loss = 0.0
    size = 0

    for data in data_loader:
        inputs, labels = data
        if use_gpu:
            inputs, labels = inputs.cuda(), labels.cuda()
            
        bs = labels.size(0)
                
        outputs = conv_class(inputs)
        loss = loss_fn(outputs, labels)
        _,preds = torch.max(outputs.data,1)
        
        # statistics
        running_loss += loss.data.item()
        running_corrects += torch.sum(preds == labels.data)
        size += bs

    print('Test - Loss: {:.4f} Acc: {:.4f}'.format(running_loss / size, running_corrects.item() / size))

In [27]:
# using simple classifier 

test(conv_class, dset_loaders['test'])

Test - Loss: 0.5008 Acc: 0.6125


In [32]:
# using vgg16  

test(net, dset_loaders['test'])

Test - Loss: 1.2838 Acc: 0.5500
